In [26]:
import os
import librosa
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import warnings 
warnings.filterwarnings("ignore")

# Define function to load and preprocess the audio data
def load_data(dataset_path, max_pad_len=174):
    data = []
    labels = []
    emotions = os.listdir(dataset_path)
    
    for emotion in emotions:
        emotion_path = os.path.join(dataset_path, emotion)
        for file in os.listdir(emotion_path):
            if file.endswith('.wav'):
                file_path = os.path.join(emotion_path, file)
                signal, sr = librosa.load(file_path, sr=22050)
                # Extract Mel-spectrogram feature
                mel_spec = librosa.feature.melspectrogram(y=signal, sr=sr)
                mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
                if mel_spec.shape[1] > max_pad_len:
                    mel_spec = mel_spec[:, :max_pad_len]
                else:
                    pad_width = max_pad_len - mel_spec.shape[1]
                    mel_spec = np.pad(mel_spec, pad_width=((0, 0), (0, pad_width)), mode='constant')
                data.append(mel_spec)
                labels.append(emotion)
    
    data = np.array(data)
    labels = np.array(labels)
    return data, labels

# Load data from the dataset directory
dataset_path = "DATASETS/Main"
data, labels = load_data(dataset_path)

# Encode labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Reshape data for the CNN model
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [19]:
# Encode labels
import pickle
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
with open('Encoder1.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [20]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Reshape data for the CNN model
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

def create_cnn_model(input_shape, num_classes):
    model = Sequential()

    # First Conv Block
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    # Second Conv Block
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    # Third Conv Block
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    # Flattening
    model.add(Flatten())

    # Fully Connected Layers
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

input_shape = (X_train.shape[1], X_train.shape[2], 1)
num_classes = len(np.unique(y_train))
model = create_cnn_model(input_shape, num_classes)

# Model Summary
model.summary()

c:\Users\sidda\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 172, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 126, 172, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 86, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 63, 86, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 84, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 61, 84, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 42, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 30, 42, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 40, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 28, 40, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 14, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 35840)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │     9,175,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,302,663 (35.49 MB)

 Trainable params: 9,302,215 (35.49 MB)

 Non-trainable params: 448 (1.75 KB)

In [22]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=32)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 31s 418ms/step - accuracy: 0.3595 - loss: 6.8411 - val_accuracy: 0.2436 - val_loss: 3.1616
Epoch 2/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 28s 409ms/step - accuracy: 0.6406 - loss: 1.0370 - val_accuracy: 0.5055 - val_loss: 1.6074
Epoch 3/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 28s 408ms/step - accuracy: 0.7485 - loss: 0.6958 - val_accuracy: 0.6073 - val_loss: 1.7596
Epoch 4/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 28s 407ms/step - accuracy: 0.8376 - loss: 0.4624 - val_accuracy: 0.8855 - val_loss: 0.3981
Epoch 5/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 29s 413ms/step - accuracy: 0.8769 - loss: 0.3573 - val_accuracy: 0.8855 - val_loss: 0.4441
Epoch 6/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 28s 412ms/step - accuracy: 0.8657 - loss: 0.3652 - val_accuracy: 0.9764 - val_loss: 0.0717
Epoch 7/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 28s 412ms/step - accuracy: 0.8802 - loss: 0.3469 - val_accuracy: 0.9600 - val_loss: 0.1672
Epoch 8/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 28s 411ms/step - accuracy: 0.9223 - loss: 0.2298 - val_accu

In [23]:
model.save('Exports/Exp1.h5')
model.save("Exports/Exp1.keras")